## Neural Matrix Factorization



In [33]:
import numpy as np
import pandas as pd
import os
import torch
import argparse
import heapq

from time import time
from scipy.sparse import load_npz
from torch import nn
from torch.optim.lr_scheduler import CyclicLR
from torch.utils.data import DataLoader, Dataset
from utils import get_train_instances, get_scores
from gmf import GMF, train, evaluate, checkpoint
from mlp import MLP

In [34]:
datadir = "."
dataname = "neuralcf_split.npz"
train_matrix = "neuralcf_train_sparse.npz"
modeldir = "models"

epochs = 1
batch_size = 512
lr = 0.01
learner = "Adam"
lr_scheduler = None

n_emb = 8

layers = [32, 16, 8]
dropouts = [0., 0.]

freeze = True

# 使用预训练好的GMF和MLP模型, 按照实际的模型名字替代如下字符串
mf_pretrain = os.path.join(modeldir, "GMF_bs_512_lr_001_n_emb_8_lrnr_adam_lrs_wolrs.pt") #"GMF_bs_1024_lr_001_n_emb_8_lrnr_adam_lrs_wolrs.pt")
mlp_pretrain = os.path.join(modeldir, "MLP_bs_512_reg_00_lr_001_n_emb_16_ll_8_dp_wodp_lrnr_adam_lrs_wolrs.pt")# "MLP_bs_1024_reg_00_lr_003_n_emb_64_ll_32_dp_wodp_lrnr_adam_lrs_wlrs.pt")

l2reg = 0.

validate_every = 1
save_model = True
n_neg = 4
topk = 10



### The Model

In [35]:
class NeuMF(nn.Module):
    def __init__(self, n_user, n_item, n_emb, layers, dropouts):
        super(NeuMF, self).__init__()

        self.layers = layers
        self.n_layers = len(layers)
        self.dropouts = dropouts
        self.n_user = n_user
        self.n_item = n_item

        self.mf_embeddings_user = nn.Embedding(n_user, n_emb)
        self.mf_embeddings_item = nn.Embedding(n_item, n_emb)

        self.mlp_embeddings_user = nn.Embedding(n_user, layers[0]//2)
        self.mlp_embeddings_item = nn.Embedding(n_item, layers[0]//2)
        self.mlp = nn.Sequential()
        for i in range(1,self.n_layers):
            self.mlp.add_module("linear%d" %i, nn.Linear(layers[i-1],layers[i]))
            self.mlp.add_module("relu%d" %i, torch.nn.ReLU())
            self.mlp.add_module("dropout%d" %i , torch.nn.Dropout(p=dropouts[i-1]))

        self.out = nn.Linear(in_features=n_emb+layers[-1], out_features=1)

        for m in self.modules():
            if isinstance(m, nn.Embedding):
                nn.init.normal_(m.weight)

    def forward(self, users, items):

        mf_user_emb = self.mf_embeddings_user(users)
        mf_item_emb = self.mf_embeddings_item(items)

        mlp_user_emb = self.mlp_embeddings_user(users)
        mlp_item_emb = self.mlp_embeddings_item(items)

        mf_emb_vector = mf_user_emb*mf_item_emb
        mlp_emb_vector = torch.cat([mlp_user_emb,mlp_item_emb], dim=1)
        mlp_emb_vector = self.mlp(mlp_emb_vector)

        emb_vector = torch.cat([mf_emb_vector,mlp_emb_vector], dim=1)
        preds = torch.sigmoid(self.out(emb_vector))

        return preds

In [36]:
my_model = NeuMF(n_user=10, n_item=10, n_emb=8, layers=layers, dropouts=dropouts)




### Load Pretrained Models

In [37]:
# Load the dataset
dataset = np.load(os.path.join(datadir, dataname))
train_ratings = load_npz(os.path.join(datadir, train_matrix)).todok()
test_ratings, negatives = dataset['test_negative'], dataset['negatives']
n_users, n_items = dataset['n_users'].item(), dataset['n_items'].item()

test_loader = DataLoader(dataset=test_ratings,
    batch_size=1000,
    shuffle=False
    )

In [38]:
# Define model structures
gmf_model = GMF(n_users, n_items, n_emb)
gmf_model.load_state_dict(torch.load(mf_pretrain))
mlp_model = MLP(n_users, n_items, layers, dropouts)
mlp_model.load_state_dict(torch.load(mlp_pretrain))

<All keys matched successfully>

打印出模型

In [39]:
gmf_model

GMF(
  (embeddings_user): Embedding(123960, 8)
  (embeddings_item): Embedding(50052, 8)
  (out): Linear(in_features=8, out_features=1, bias=True)
)

In [40]:
mlp_model

MLP(
  (embeddings_user): Embedding(123960, 16)
  (embeddings_item): Embedding(50052, 16)
  (mlp): Sequential(
    (linear1): Linear(in_features=32, out_features=16, bias=True)
    (relu1): ReLU()
    (dropout1): Dropout(p=0.0, inplace=False)
    (linear2): Linear(in_features=16, out_features=8, bias=True)
    (relu2): ReLU()
    (dropout2): Dropout(p=0.0, inplace=False)
  )
  (out): Linear(in_features=8, out_features=1, bias=True)
)

In [41]:
model = NeuMF(n_users, n_items, n_emb, layers, dropouts)
model

NeuMF(
  (mf_embeddings_user): Embedding(123960, 8)
  (mf_embeddings_item): Embedding(50052, 8)
  (mlp_embeddings_user): Embedding(123960, 16)
  (mlp_embeddings_item): Embedding(50052, 16)
  (mlp): Sequential(
    (linear1): Linear(in_features=32, out_features=16, bias=True)
    (relu1): ReLU()
    (dropout1): Dropout(p=0.0, inplace=False)
    (linear2): Linear(in_features=16, out_features=8, bias=True)
    (relu2): ReLU()
    (dropout2): Dropout(p=0.0, inplace=False)
  )
  (out): Linear(in_features=16, out_features=1, bias=True)
)

加载模型到 NeuMF model 

In [42]:
# GMF embeddings
model.mf_embeddings_item.weight = gmf_model.embeddings_item.weight
model.mf_embeddings_user.weight = gmf_model.embeddings_user.weight

In [43]:
# MLP embeddings
model.mlp_embeddings_item.weight = mlp_model.embeddings_item.weight
model.mlp_embeddings_user.weight = mlp_model.embeddings_user.weight

In [44]:
# MLP layers
model_dict = model.state_dict()
mlp_layers_dict = mlp_model.state_dict()
mlp_layers_dict = {k: v for k, v in mlp_layers_dict.items() if 'linear' in k}
model_dict.update(mlp_layers_dict)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [45]:
# Prediction weights
mf_prediction_weight, mf_prediction_bias = gmf_model.out.weight, gmf_model.out.bias
mlp_prediction_weight, mlp_prediction_bias = mlp_model.out.weight, mlp_model.out.bias

new_weight = torch.cat([mf_prediction_weight, mlp_prediction_weight], dim=1)
new_bias = mf_prediction_bias + mlp_prediction_bias
model.out.weight = torch.nn.Parameter(0.5*new_weight)
model.out.bias = torch.nn.Parameter(0.5*new_bias)



### Freeze all up to Last (output) Layer



In [46]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()

if freeze:
    for name, layer in model.named_parameters():
        if not ("out" in name):
            layer.requires_grad = False
# or this and pass train_parametes to the optimizer
# train_parametes = model.out.parameters() if freeze else model.parameters()

### Run the Model

In [47]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2reg)

criterion = nn.BCELoss()

scheduler = None

# let's make sure all is ok
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
trainable_params = sum([np.prod(p.size()) for p in model_parameters])
print(trainable_params)

17


All ok, 40 (32+8) weights + a bias is all that we will be training here

In [48]:
best_hr, best_ndcgm, best_iter=0,0,0
for epoch in range(1,epochs+1):
    t1 = time()
    loss = train(model, criterion, optimizer, scheduler, epoch, batch_size,
        use_cuda, train_ratings, negatives, n_items, n_neg)
    t2 = time()
    if epoch % validate_every == 0:
        (hr, ndcg) = evaluate(model, test_loader, use_cuda, topk)
        print("Epoch: {} {:.2f}s, LOSS = {:.4f}, HR = {:.4f}, NDCG = {:.4f}, validated in {:.2f}s".
            format(epoch, t2-t1, loss, hr, ndcg, time()-t2))

Epoch: 1 135.22s, LOSS = 0.3660, HR = 0.5330, NDCG = 0.3274, validated in 98.74s
